# DrivenData Water Pump Competition
- Parts of this analysis were taken from/inspired by this blog post: <https://www.civisanalytics.com/blog/workflows-in-python-getting-data-ready-to-build-models/>

In [233]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

In [234]:
features_df = pd.read_csv('data/training.csv')
features_df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [235]:
labels_df = pd.read_csv('data/training_labels.csv')
labels_df.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


The goal is to predict the well status. There are 3 categories: functional, non-functional, and functional but needs repair. The 'functional but needs repair' group is much smaller than the other two. Might need to take this into account when evaluating our classifier. The majority class in the training data is 'functional'; just predicting this would give an accuracy of 54%.

In [236]:
labels_df.status_group.value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [237]:
# What accuracy would we get by just predicting majority class?
sum(labels_df.status_group=='functional') / len(labels_df)

0.54308080808080805

In [238]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 2 columns):
id              59400 non-null int64
status_group    59400 non-null object
dtypes: int64(1), object(1)
memory usage: 928.2+ KB


Firt we will encode the status_group as 0,1,2 for modeling

In [239]:
def label_map(x):
    if x=='functional':
        return 2
    elif x=='non functional':
        return 1
    elif x=='functional needs repair':
        return 0

# apply to labels now
labels_df['status_group'] = labels_df['status_group'].apply(label_map)
labels_df.head()

,id,status_group
0,69572,2
1,8776,2
2,34310,2
3,67743,1
4,19728,2


Let's take a closer look at the features

In [240]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
r

Which features have missing/null values? You can see from df.info() which variables have nulls (counts are less than total # rows), but it's nice to do this more systematically when you have a lot of features.
- Below we see that we have 7 features that have missing/null values.

In [241]:
null_counts = features_df.isnull().sum()
null_counts = null_counts[null_counts>0]
null_counts.sort_values(ascending=False)

scheme_name          28166
scheme_management     3877
installer             3655
funder                3635
public_meeting        3334
permit                3056
subvillage             371
dtype: int64

In [242]:
#features_df.scheme_name.value_counts(dropna=False)

In [243]:
#features_df.scheme_management.value_counts(dropna=False)

In [244]:
#features_df.installer.value_counts(dropna=False)

In [245]:
#features_df.funder.value_counts(dropna=False)

In [246]:
#features_df.public_meeting.value_counts(dropna=False)

In [247]:
#features_df.permit.value_counts(dropna=False)

In [248]:
#features_df.subvillage.value_counts(dropna=False)

Summary of dealing with missing values:
- permit: Use most common
- public_meeting : Use most common

Drop these, and possibly try to do something w/ them later:
- scheme_name
- scheme_management
- installers
- funder
- subvillage

Other modifications:
- date_recorded has way too many values to encode. Drop for now. Maybe try using month or year later?
- wpt_name has 37400 unique values! Drop this
- ward has over 2000 unique values

In [249]:
# fill in missing values in permit_value and public_meeint
features_df.permit.value_counts(dropna=False)
#features_df['permit'].loc[features_df.permit.isnull()]=True
features_df.fillna({'permit':True,'public_meeting':True},inplace=True)
features_df.permit.value_counts(dropna=False)


True     41908
False    17492
Name: permit, dtype: int64

In [250]:
feat_to_drop = ['id','date_recorded','ward','wpt_name','scheme_name','scheme_management','installer','funder','subvillage']
features_df.drop(feat_to_drop,axis=1,inplace=True)
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 31 columns):
amount_tsh               59400 non-null float64
gps_height               59400 non-null int64
longitude                59400 non-null float64
latitude                 59400 non-null float64
num_private              59400 non-null int64
basin                    59400 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
population               59400 non-null int64
public_meeting           59400 non-null bool
recorded_by              59400 non-null object
permit                   59400 non-null bool
construction_year        59400 non-null int64
extraction_type          59400 non-null object
extraction_type_group    59400 non-null object
extraction_type_class    59400 non-null object
management               59400 non-null object
manag

In [251]:
non_num_cols = features_df.select_dtypes(['object']).columns
non_num_cols
for col in non_num_cols:
    print(col)
    print(features_df[col].value_counts().count())

basin
9
region
21
lga
125
recorded_by
1
extraction_type
18
extraction_type_group
13
extraction_type_class
7
management
12
management_group
5
payment
7
payment_type
7
water_quality
8
quality_group
6
quantity
5
quantity_group
5
source
10
source_type
7
source_class
3
waterpoint_type
7
waterpoint_type_group
6


Next we need to encode all the non-numeric features for modeling.

First make a list of how many distinct values each categorical feature has; we don't wont to create too many new dummy variables.

In [252]:
#non_num_features = features_df.columns[features_df.columns.dtype=='object']
#non_num_features
non_num_df = features_df.select_dtypes(['object'])
#non_num_df
dums = pd.get_dummies(non_num_df,drop_first=True)
dums.shape

(59400, 262)

Combine dummied features w/ the numeric features

In [253]:
import numpy
num_df = features_df.select_dtypes([numpy.number])
num_df.head()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
0,6000.0,1390,34.938093,-9.856322,0,11,5,109,1999
1,0.0,1399,34.698766,-2.147466,0,20,2,280,2010
2,25.0,686,37.460664,-3.821329,0,21,4,250,2009
3,0.0,263,38.486161,-11.155298,0,90,63,58,1986
4,0.0,0,31.130847,-1.825359,0,18,1,0,0


We also need to encode the categorical features for modeling.

In [254]:
X = pd.concat([num_df,dums],axis=1)
X.shape

(59400, 271)

In [204]:
X.head()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,basin_Lake Nyasa,...,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,waterpoint_type_group_communal standpipe,waterpoint_type_group_dam,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other
0,6000.0,1390,34.938093,-9.856322,0,11,5,109,1999,1,...,0,0,0,0,0,1,0,0,0,0
1,0.0,1399,34.698766,-2.147466,0,20,2,280,2010,0,...,0,0,0,0,0,1,0,0,0,0
2,25.0,686,37.460664,-3.821329,0,21,4,250,2009,0,...,1,0,0,0,0,1,0,0,0,0
3,0.0,263,38.486161,-11.155298,0,90,63,58,1986,0,...,1,0,0,0,0,1,0,0,0,0
4,0.0,0,31.130847,-1.825359,0,18,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [205]:
#X = features_df.drop('id',axis=1)
#X = 
y = labels_df.drop('id',axis=1).values

In [206]:
#X.shape
#y.shape
#y.head()
y=y.ravel() # reshape for model


In [207]:
y.shape

(59400,)

### split into training/testing set

In [208]:
 from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

### logistic regression

In [209]:
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
clf = LogisticRegression()
np.mean(cross_val_score(clf,X_train,y_train))

0.71600470006925165

In [210]:
clf = LogisticRegressionCV()
clf.fit(X_train,y_train)
#np.mean(cross_val_score(clf,X_train,y_train))
clf.score(X_train,y_train)
clf.score(X_test,y_test)

0.70740740740740737

### random forest

In [211]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
np.mean(cross_val_score(rf,X_train,y_train))

0.773894564867882

In [33]:
# test-set accuracy
rf.score(X_test,y_test)

0.78114478114478114

In [212]:
# Try to optimize
from sklearn.model_selection import GridSearchCV
#params = {'n_estimators':[125,150,200]}
params = {'min_samples_split':[5,10,15]}
rf = RandomForestClassifier(n_estimators=150,n_jobs=-1)
cv = GridSearchCV(rf,params)
cv.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=150, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'min_samples_split': [5, 10, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [213]:
cv.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=10, min_weight_fraction_leaf=0.0,
            n_estimators=150, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [214]:
np.mean(cross_val_score(cv.best_estimator_,X_train,y_train))

0.79602693087693899

### Feature importances

In [44]:
imp = pd.DataFrame({'vars':X.columns,'imp':cv.best_estimator_.feature_importances_})
imp.sort_values('imp',ascending=False,inplace=True)
imp

,imp,vars
3,0.139518,latitude
2,0.137203,longitude
1,0.066191,gps_height
8,0.050216,construction_year
7,0.046421,population
237,0.023896,quantity_enough
241,0.021265,quantity_group_enough
0,0.020694,amount_tsh
272,0.018617,waterpoint_type_group_other
6,0.018530,district_code


### Make predictions to submit!

In [279]:
test_df = pd.read_csv('data/test.csv')
test_df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [280]:
#test_df['extraction_typeother - mkulima/shinyanga ']
#X.extraction_type_
#Xtest.extraction_type_

Now do same modifications to the test data:

In [281]:
def prep_data(dfin):
    df=dfin.copy()
    df.permit.value_counts(dropna=False)
    df.fillna({'permit':True,'public_meeting':True},inplace=True)
    feat_to_drop = ['id','date_recorded','ward','wpt_name','scheme_name','scheme_management','installer','funder','subvillage']
    df.drop(feat_to_drop,axis=1,inplace=True)
    non_num_df = df.select_dtypes(['object'])
    dums = pd.get_dummies(non_num_df,drop_first=True)
    num_df = df.select_dtypes([numpy.number])
    num_df.head()
    X = pd.concat([num_df,dums],axis=1)
    return X



In [282]:
Xtest = prep_data(test_df)
Xtest.head()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,basin_Lake Nyasa,...,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,waterpoint_type_group_communal standpipe,waterpoint_type_group_dam,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other
0,0.0,1996,35.290799,-4.059696,0,21,3,321,2012,0,...,0,0,0,0,1,0,0,0,0,1
1,0.0,1569,36.656709,-3.309214,0,2,2,300,2000,0,...,0,0,0,0,0,1,0,0,0,0
2,0.0,1567,34.767863,-5.004344,0,13,2,500,2010,0,...,0,0,0,0,1,0,0,0,0,1
3,0.0,267,38.058046,-9.418672,0,80,43,250,1987,0,...,0,0,0,0,1,0,0,0,0,1
4,500.0,1260,35.006123,-10.950412,0,10,3,60,2000,0,...,0,0,0,0,0,1,0,0,0,0


In [283]:
Xtest['extraction_type_other - mkulima/shinyanga']=0
X.shape

(59400, 271)

In [284]:
#print(X.columns,Xtest.columns)
#X.columns==Xtest.columns
for i in range(0,len(X)):
    print(X.columns[i],' : ' ,Xtest.columns[i])

amount_tsh  :  amount_tsh
gps_height  :  gps_height
longitude  :  longitude
latitude  :  latitude
num_private  :  num_private
region_code  :  region_code
district_code  :  district_code
population  :  population
construction_year  :  construction_year
basin_Lake Nyasa  :  basin_Lake Nyasa
basin_Lake Rukwa  :  basin_Lake Rukwa
basin_Lake Tanganyika  :  basin_Lake Tanganyika
basin_Lake Victoria  :  basin_Lake Victoria
basin_Pangani  :  basin_Pangani
basin_Rufiji  :  basin_Rufiji
basin_Ruvuma / Southern Coast  :  basin_Ruvuma / Southern Coast
basin_Wami / Ruvu  :  basin_Wami / Ruvu
region_Dar es Salaam  :  region_Dar es Salaam
region_Dodoma  :  region_Dodoma
region_Iringa  :  region_Iringa
region_Kagera  :  region_Kagera
region_Kigoma  :  region_Kigoma
region_Kilimanjaro  :  region_Kilimanjaro
region_Lindi  :  region_Lindi
region_Manyara  :  region_Manyara
region_Mara  :  region_Mara
region_Mbeya  :  region_Mbeya
region_Morogoro  :  region_Morogoro
region_Mtwara  :  region_Mtwara
region_M

IndexError: index 271 is out of bounds for axis 0 with size 271

In the test data, there is a vlue for extraction_type that was not present in the training data...

In [285]:
#test_df.extraction_type.value_counts()
Xtest.extra

AttributeError: 'DataFrame' object has no attribute 'extra'

In [286]:
test_pred = cv.best_estimator_.predict(Xtest)

In [287]:
test_pred

array([2, 2, 2, ..., 2, 2, 1])

In [290]:
# write to csv
df_preds = pd.DataFrame({'id':test_df.id,'status_group':test_pred})
df_preds.head()

,id,status_group
0,50785,2
1,51630,2
2,17168,2
3,45559,1
4,49871,1


In [294]:
# map back to labels for output
def label_map_rev(x):
    if x==2:
        return 'functional'
    elif x==1:
        return 'non functional'
    elif x==0:
        return 'functional needs repair'

# apply to labels now
df_preds['status_group'] = df_preds['status_group'].apply(label_map_rev)
df_preds.head()

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,non functional


In [295]:
df_preds.to_csv('testout.csv',index=False)

In [ ]:
plt.scatter(features_df.latitude, labels_df.status_group)

In [ ]:
import seaborn as sns
sns.boxplot(labels_df.status_group,features_df.latitude)

In [ ]:
sns.boxplot(labels_df.status_group, features_df.gps_height)

In [ ]:
# try SVM?


In [ ]:
# try gradient boosted tree?

